In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/날씨빅데이터티이임/최종데이터.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1578960 entries, 0 to 1578959
Data columns (total 32 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Unnamed: 0   1578960 non-null  int64  
 1   날짜           1578960 non-null  int64  
 2   날짜_연도        1578960 non-null  int64  
 3   날짜_월         1578960 non-null  int64  
 4   날짜_일         1578960 non-null  int64  
 5   시간_분         1578960 non-null  int64  
 6   관측소지점        1578960 non-null  int64  
 7   경도           1578960 non-null  float64
 8   위도           1578960 non-null  float64
 9   자외선지수        1578960 non-null  float64
 10  파랑_가시밴드      1578960 non-null  float64
 11  초록_가시밴드      1578960 non-null  float64
 12  빨강_가시밴드      1578960 non-null  float64
 13  식생_가시밴드      1578960 non-null  float64
 14  권운_밴드        1578960 non-null  float64
 15  눈/얼음_가시밴드    1578960 non-null  float64
 16  야간안개/하층운_밴드  1578960 non-null  float64
 17  상층_수증기_밴드    1578960 non-null  float64
 18  중층

In [ ]:
df_1 = df[(df['날짜_월']==5)|(df['날짜_월']==6)|(df['날짜_월']==7)]

In [ ]:
df1 = df_1.drop(['날짜', '날짜_연도', '날짜_월', '날짜_일', '시간_분', '경도', '위도', '위성천정각', '관측고도', '계절'], axis=1)

In [ ]:
df1.reset_index(drop=True, inplace = True)

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397440 entries, 0 to 397439
Data columns (total 22 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   397440 non-null  int64  
 1   관측소지점        397440 non-null  int64  
 2   자외선지수        397440 non-null  float64
 3   파랑_가시밴드      397440 non-null  float64
 4   초록_가시밴드      397440 non-null  float64
 5   빨강_가시밴드      397440 non-null  float64
 6   식생_가시밴드      397440 non-null  float64
 7   권운_밴드        397440 non-null  float64
 8   눈/얼음_가시밴드    397440 non-null  float64
 9   야간안개/하층운_밴드  397440 non-null  float64
 10  상층_수증기_밴드    397440 non-null  float64
 11  중층_수증기_밴드    397440 non-null  float64
 12  하층_수증기_밴드    397440 non-null  float64
 13  구름상_밴드       397440 non-null  float64
 14  오존_밴드        397440 non-null  float64
 15  대기창_밴드       397440 non-null  float64
 16  깨끗한_대기창_밴드   397440 non-null  float64
 17  오염된_대기창_밴드   397440 non-null  float64
 18  이산화탄소_밴드     397440 non-

In [ ]:
del df1['Unnamed: 0']

In [ ]:
pd.set_option('display.max_columns', 30)

In [ ]:
df1.head()

,관측소지점,자외선지수,파랑_가시밴드,초록_가시밴드,빨강_가시밴드,식생_가시밴드,권운_밴드,눈/얼음_가시밴드,야간안개/하층운_밴드,상층_수증기_밴드,중층_수증기_밴드,하층_수증기_밴드,구름상_밴드,오존_밴드,대기창_밴드,깨끗한_대기창_밴드,오염된_대기창_밴드,이산화탄소_밴드,태양천정각,대기외일사량,지면타입
0,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,277.64337,241.10364,252.81774,260.87956,276.05914,252.91737,277.63245,277.03649,275.67037,262.71147,130.35092,-6.11462,0
1,105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,287.64354,237.44891,246.82269,256.06092,284.76266,256.69212,286.82011,287.23821,285.51819,269.40510,126.43187,-5.60867,3
2,108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,277.32743,238.35866,248.32240,257.48046,278.59032,253.85590,281.28558,281.42512,280.03692,266.81080,126.31564,-5.59822,3
3,112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,278.71774,238.44819,248.14894,257.41386,280.26998,254.78101,283.03827,283.39275,282.06360,268.05385,126.36437,-5.60546,3
4,115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,281.88250,233.40607,241.62566,250.50873,275.90815,251.68704,277.82840,278.25056,275.71853,262.56565,126.91833,-5.66707,4


In [ ]:
# 파랑, 초록, 빨강, 식생 가시밴드 (단위x)
df1['Band1'] = df1.iloc[:, 2:6].mean(axis=1)
# 권운, 눈/얼음 가시밴드 (단위x)
df1['Band2'] = df1.iloc[:,  6:8].mean(axis=1)
# 단파적외채널(야간안개/하층운, 단위 K(캘빈))
df1['Band3'] = df1.iloc[:, 8:9].mean(axis=1)
# 수증기채널(상,중,하, 단위 K(캘빈))
df1['Band4'] = df1.iloc[:, 9:12].mean(axis=1)
# 적외채널(구름상, 오존, 대기창, 깨끗, 오염된 대기창, 단위 K(캘빈))
df1['Band5'] = df1.iloc[:, 12:18].mean(axis=1)

In [ ]:
df1['태양천정각'].max()

131.63541

In [ ]:
bins = list(range(0, 181, 45))
bins_label = list(range(1,6,1))
df1["time"] = pd.cut(df1['태양천정각'], bins, right=False, labels=bins_label[:-1])

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397440 entries, 0 to 397439
Data columns (total 27 columns):
 #   Column       Non-Null Count   Dtype   
---  ------       --------------   -----   
 0   관측소지점        397440 non-null  int64   
 1   자외선지수        397440 non-null  float64 
 2   파랑_가시밴드      397440 non-null  float64 
 3   초록_가시밴드      397440 non-null  float64 
 4   빨강_가시밴드      397440 non-null  float64 
 5   식생_가시밴드      397440 non-null  float64 
 6   권운_밴드        397440 non-null  float64 
 7   눈/얼음_가시밴드    397440 non-null  float64 
 8   야간안개/하층운_밴드  397440 non-null  float64 
 9   상층_수증기_밴드    397440 non-null  float64 
 10  중층_수증기_밴드    397440 non-null  float64 
 11  하층_수증기_밴드    397440 non-null  float64 
 12  구름상_밴드       397440 non-null  float64 
 13  오존_밴드        397440 non-null  float64 
 14  대기창_밴드       397440 non-null  float64 
 15  깨끗한_대기창_밴드   397440 non-null  float64 
 16  오염된_대기창_밴드   397440 non-null  float64 
 17  이산화탄소_밴드     397440 non-null  float64 
 18  태양천정

In [ ]:
df_a = df1.copy()
df_a = pd.get_dummies(df_a, columns = ['관측소지점'])
df_a = pd.get_dummies(df_a, columns = ['지면타입'])
df_a = pd.get_dummies(df_a, columns = ['time'])

In [ ]:
df_a.reset_index(drop = True, inplace = True)

In [ ]:
df_a

,자외선지수,파랑_가시밴드,초록_가시밴드,빨강_가시밴드,식생_가시밴드,권운_밴드,눈/얼음_가시밴드,야간안개/하층운_밴드,상층_수증기_밴드,중층_수증기_밴드,하층_수증기_밴드,구름상_밴드,오존_밴드,대기창_밴드,깨끗한_대기창_밴드,...,관측소지점_138,관측소지점_143,관측소지점_146,관측소지점_152,관측소지점_156,관측소지점_159,관측소지점_165,지면타입_0,지면타입_2,지면타입_3,지면타입_4,time_1,time_2,time_3,time_4
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,277.64337,241.10364,252.81774,260.87956,276.05914,252.91737,277.63245,277.03649,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,287.64354,237.44891,246.82269,256.06092,284.76266,256.69212,286.82011,287.23821,...,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,277.32743,238.35866,248.32240,257.48046,278.59032,253.85590,281.28558,281.42512,...,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,278.71774,238.44819,248.14894,257.41386,280.26998,254.78101,283.03827,283.39275,...,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,281.88250,233.40607,241.62566,250.50873,275.90815,251.68704,277.82840,278.25056,...,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397435,0.1,0.0,0.0,0.0,0.0,0.0,0.0,296.40523,236.12825,244.63256,253.42097,289.04391,264.53165,291.90287,290.09587,...,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0
397436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,290.86107,235.64237,244.82259,253.63901,285.61006,262.88449,288.27382,286.67481,...,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
397437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,292.92384,236.60649,245.04310,253.49377,285.63638,262.57588,287.50208,284.96722,...,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0
397438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,286.67795,236.41609,246.52170,255.96927,284.69604,262.22936,287.43410,286.66069,...,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0


In [ ]:
df_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397440 entries, 0 to 397439
Data columns (total 47 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   자외선지수        397440 non-null  float64
 1   파랑_가시밴드      397440 non-null  float64
 2   초록_가시밴드      397440 non-null  float64
 3   빨강_가시밴드      397440 non-null  float64
 4   식생_가시밴드      397440 non-null  float64
 5   권운_밴드        397440 non-null  float64
 6   눈/얼음_가시밴드    397440 non-null  float64
 7   야간안개/하층운_밴드  397440 non-null  float64
 8   상층_수증기_밴드    397440 non-null  float64
 9   중층_수증기_밴드    397440 non-null  float64
 10  하층_수증기_밴드    397440 non-null  float64
 11  구름상_밴드       397440 non-null  float64
 12  오존_밴드        397440 non-null  float64
 13  대기창_밴드       397440 non-null  float64
 14  깨끗한_대기창_밴드   397440 non-null  float64
 15  오염된_대기창_밴드   397440 non-null  float64
 16  이산화탄소_밴드     397440 non-null  float64
 17  태양천정각        397440 non-null  float64
 18  대기외일사량       397440 non-

In [ ]:
df_a['Band345'] = df_a.iloc[:,21:24].mean(axis=1)

In [ ]:
df_a.reset_index(drop=True,inplace = True)

In [ ]:
df_a = df_a.drop(['대기외일사량'],axis=1)

In [ ]:
df_a = df_a.drop(['time_4'], axis=1)

In [ ]:
df_a

,자외선지수,파랑_가시밴드,초록_가시밴드,빨강_가시밴드,식생_가시밴드,권운_밴드,눈/얼음_가시밴드,야간안개/하층운_밴드,상층_수증기_밴드,중층_수증기_밴드,하층_수증기_밴드,구름상_밴드,오존_밴드,대기창_밴드,깨끗한_대기창_밴드,...,관측소지점_138,관측소지점_143,관측소지점_146,관측소지점_152,관측소지점_156,관측소지점_159,관측소지점_165,지면타입_0,지면타입_2,지면타입_3,지면타입_4,time_1,time_2,time_3,Band345
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,277.64337,241.10364,252.81774,260.87956,276.05914,252.91737,277.63245,277.03649,...,0,0,0,0,0,0,0,1,0,0,0,0,0,1,266.527188
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,287.64354,237.44891,246.82269,256.06092,284.76266,256.69212,286.82011,287.23821,...,0,0,0,0,0,0,0,0,0,1,0,0,0,1,270.942371
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,277.32743,238.35866,248.32240,257.48046,278.59032,253.85590,281.28558,281.42512,...,0,0,0,0,0,0,0,0,0,1,0,0,0,1,266.349570
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,278.71774,238.44819,248.14894,257.41386,280.26998,254.78101,283.03827,283.39275,...,0,0,0,0,0,0,0,0,0,1,0,0,0,1,267.329327
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,281.88250,233.40607,241.62566,250.50873,275.90815,251.68704,277.82840,278.25056,...,0,0,0,0,0,0,0,0,0,0,1,0,0,1,264.685236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397435,0.1,0.0,0.0,0.0,0.0,0.0,0.0,296.40523,236.12825,244.63256,253.42097,289.04391,264.53165,291.90287,290.09587,...,0,0,1,0,0,0,0,0,0,1,0,0,0,1,274.371378
397436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,290.86107,235.64237,244.82259,253.63901,285.61006,262.88449,288.27382,286.67481,...,0,0,0,1,0,0,0,0,1,0,0,0,0,1,271.583841
397437,0.0,0.0,0.0,0.0,0.0,0.0,0.0,292.92384,236.60649,245.04310,253.49377,285.63638,262.57588,287.50208,284.96722,...,0,0,0,0,1,0,0,0,0,1,0,0,0,1,272.007159
397438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,286.67795,236.41609,246.52170,255.96927,284.69604,262.22936,287.43410,286.66069,...,0,0,0,0,0,1,0,0,0,1,0,0,0,1,270.730265


In [ ]:
df_a.columns

Index(['자외선지수', '파랑_가시밴드', '초록_가시밴드', '빨강_가시밴드', '식생_가시밴드', '권운_밴드',
       '눈/얼음_가시밴드', '야간안개/하층운_밴드', '상층_수증기_밴드', '중층_수증기_밴드', '하층_수증기_밴드',
       '구름상_밴드', '오존_밴드', '대기창_밴드', '깨끗한_대기창_밴드', '오염된_대기창_밴드', '이산화탄소_밴드',
       '태양천정각', 'Band1', 'Band2', 'Band3', 'Band4', 'Band5', '관측소지점_13',
       '관측소지점_105', '관측소지점_108', '관측소지점_112', '관측소지점_115', '관측소지점_131',
       '관측소지점_132', '관측소지점_133', '관측소지점_138', '관측소지점_143', '관측소지점_146',
       '관측소지점_152', '관측소지점_156', '관측소지점_159', '관측소지점_165', '지면타입_0', '지면타입_2',
       '지면타입_3', '지면타입_4', 'time_1', 'time_2', 'time_3', 'Band345'],
      dtype='object')

In [ ]:
df_f = df_a.drop(['Band3','Band4','Band5', '파랑_가시밴드','초록_가시밴드','빨강_가시밴드','식생_가시밴드','권운_밴드','눈/얼음_가시밴드',
                 '야간안개/하층운_밴드','상층_수증기_밴드','중층_수증기_밴드','하층_수증기_밴드','구름상_밴드','오존_밴드',
              '대기창_밴드','깨끗한_대기창_밴드','오염된_대기창_밴드','이산화탄소_밴드'], axis=1)

In [ ]:
df_f

,자외선지수,태양천정각,Band1,Band2,관측소지점_13,관측소지점_105,관측소지점_108,관측소지점_112,관측소지점_115,관측소지점_131,관측소지점_132,관측소지점_133,관측소지점_138,관측소지점_143,관측소지점_146,관측소지점_152,관측소지점_156,관측소지점_159,관측소지점_165,지면타입_0,지면타입_2,지면타입_3,지면타입_4,time_1,time_2,time_3,Band345
0,0.0,130.35092,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,266.527188
1,0.0,126.43187,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,270.942371
2,0.0,126.31564,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,266.349570
3,0.0,126.36437,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,267.329327
4,0.0,126.91833,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,264.685236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397435,0.1,124.36364,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,274.371378
397436,0.0,125.09137,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,271.583841
397437,0.0,124.95628,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,272.007159
397438,0.0,125.49764,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,270.730265


In [ ]:
df_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397440 entries, 0 to 397439
Data columns (total 27 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   자외선지수      397440 non-null  float64
 1   태양천정각      397440 non-null  float64
 2   Band1      397440 non-null  float64
 3   Band2      397440 non-null  float64
 4   관측소지점_13   397440 non-null  uint8  
 5   관측소지점_105  397440 non-null  uint8  
 6   관측소지점_108  397440 non-null  uint8  
 7   관측소지점_112  397440 non-null  uint8  
 8   관측소지점_115  397440 non-null  uint8  
 9   관측소지점_131  397440 non-null  uint8  
 10  관측소지점_132  397440 non-null  uint8  
 11  관측소지점_133  397440 non-null  uint8  
 12  관측소지점_138  397440 non-null  uint8  
 13  관측소지점_143  397440 non-null  uint8  
 14  관측소지점_146  397440 non-null  uint8  
 15  관측소지점_152  397440 non-null  uint8  
 16  관측소지점_156  397440 non-null  uint8  
 17  관측소지점_159  397440 non-null  uint8  
 18  관측소지점_165  397440 non-null  uint8  
 19  지면타입_0     397440 non-n

In [ ]:
df_d = pd.read_csv('/content/drive/Shareddrives/날씨빅데이터티이임/검증/검증데이터_전처리O.csv',index_col=0)

In [ ]:
df_d

,관측소지점,자외선지수,태양천정각,지면타입,Band1,Band2,Band3,Band4,Band5,time
0,13,NaN,123.72706,0,0.0,0.0,262.92059,234.949597,247.728232,3
1,105,NaN,119.74915,3,0.0,0.0,276.10419,233.824097,255.793588,3
2,108,NaN,119.65871,3,0.0,0.0,282.27548,238.830430,264.089318,3
3,112,NaN,119.71313,3,0.0,0.0,282.46967,239.530947,265.642928,3
4,115,NaN,120.21661,4,0.0,0.0,269.13950,231.664013,248.037057,3
...,...,...,...,...,...,...,...,...,...,...
51850,146,NaN,119.84431,3,0.0,0.0,283.16913,252.730927,278.932180,3
51851,152,NaN,120.42121,2,0.0,0.0,293.89906,255.559123,284.569388,3
51852,156,NaN,120.46896,3,0.0,0.0,287.05774,255.778033,281.767140,3
51853,159,NaN,120.85844,3,0.0,0.0,286.45840,249.756860,279.538762,3


In [ ]:
df_d = pd.get_dummies(df_d, columns=['관측소지점','지면타입','time'])

In [ ]:
df_d

,자외선지수,태양천정각,Band1,Band2,Band3,Band4,Band5,관측소지점_13,관측소지점_105,관측소지점_108,관측소지점_112,관측소지점_115,관측소지점_131,관측소지점_132,관측소지점_133,관측소지점_138,관측소지점_143,관측소지점_146,관측소지점_152,관측소지점_156,관측소지점_159,관측소지점_165,지면타입_0,지면타입_2,지면타입_3,지면타입_4,time_1,time_2,time_3
0,NaN,123.72706,0.0,0.0,262.92059,234.949597,247.728232,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,NaN,119.74915,0.0,0.0,276.10419,233.824097,255.793588,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,NaN,119.65871,0.0,0.0,282.27548,238.830430,264.089318,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,NaN,119.71313,0.0,0.0,282.46967,239.530947,265.642928,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,NaN,120.21661,0.0,0.0,269.13950,231.664013,248.037057,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51850,NaN,119.84431,0.0,0.0,283.16913,252.730927,278.932180,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
51851,NaN,120.42121,0.0,0.0,293.89906,255.559123,284.569388,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
51852,NaN,120.46896,0.0,0.0,287.05774,255.778033,281.767140,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1
51853,NaN,120.85844,0.0,0.0,286.45840,249.756860,279.538762,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1


In [ ]:
df_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51855 entries, 0 to 51854
Data columns (total 29 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   자외선지수      0 non-null      float64
 1   태양천정각      51855 non-null  float64
 2   Band1      51855 non-null  float64
 3   Band2      51855 non-null  float64
 4   Band3      51855 non-null  float64
 5   Band4      51855 non-null  float64
 6   Band5      51855 non-null  float64
 7   관측소지점_13   51855 non-null  uint8  
 8   관측소지점_105  51855 non-null  uint8  
 9   관측소지점_108  51855 non-null  uint8  
 10  관측소지점_112  51855 non-null  uint8  
 11  관측소지점_115  51855 non-null  uint8  
 12  관측소지점_131  51855 non-null  uint8  
 13  관측소지점_132  51855 non-null  uint8  
 14  관측소지점_133  51855 non-null  uint8  
 15  관측소지점_138  51855 non-null  uint8  
 16  관측소지점_143  51855 non-null  uint8  
 17  관측소지점_146  51855 non-null  uint8  
 18  관측소지점_152  51855 non-null  uint8  
 19  관측소지점_156  51855 non-null  uint8  
 20  관측소지점_

In [ ]:
df_d['Band345'] = df_d.iloc[:,4:7].mean(axis=1)

In [ ]:
df_d

,자외선지수,태양천정각,Band1,Band2,Band3,Band4,Band5,관측소지점_13,관측소지점_105,관측소지점_108,관측소지점_112,관측소지점_115,관측소지점_131,관측소지점_132,관측소지점_133,관측소지점_138,관측소지점_143,관측소지점_146,관측소지점_152,관측소지점_156,관측소지점_159,관측소지점_165,지면타입_0,지면타입_2,지면타입_3,지면타입_4,time_1,time_2,time_3,Band345
0,NaN,123.72706,0.0,0.0,262.92059,234.949597,247.728232,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,248.532806
1,NaN,119.74915,0.0,0.0,276.10419,233.824097,255.793588,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,255.240625
2,NaN,119.65871,0.0,0.0,282.27548,238.830430,264.089318,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,261.731743
3,NaN,119.71313,0.0,0.0,282.46967,239.530947,265.642928,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,262.547848
4,NaN,120.21661,0.0,0.0,269.13950,231.664013,248.037057,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,249.613523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51850,NaN,119.84431,0.0,0.0,283.16913,252.730927,278.932180,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,271.610746
51851,NaN,120.42121,0.0,0.0,293.89906,255.559123,284.569388,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,278.009191
51852,NaN,120.46896,0.0,0.0,287.05774,255.778033,281.767140,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,274.867638
51853,NaN,120.85844,0.0,0.0,286.45840,249.756860,279.538762,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,271.918007


In [ ]:
df_d = df_d.drop(['Band3', 'Band4', 'Band5'], axis=1)

In [ ]:
df_d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51855 entries, 0 to 51854
Data columns (total 27 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   자외선지수      0 non-null      float64
 1   태양천정각      51855 non-null  float64
 2   Band1      51855 non-null  float64
 3   Band2      51855 non-null  float64
 4   관측소지점_13   51855 non-null  uint8  
 5   관측소지점_105  51855 non-null  uint8  
 6   관측소지점_108  51855 non-null  uint8  
 7   관측소지점_112  51855 non-null  uint8  
 8   관측소지점_115  51855 non-null  uint8  
 9   관측소지점_131  51855 non-null  uint8  
 10  관측소지점_132  51855 non-null  uint8  
 11  관측소지점_133  51855 non-null  uint8  
 12  관측소지점_138  51855 non-null  uint8  
 13  관측소지점_143  51855 non-null  uint8  
 14  관측소지점_146  51855 non-null  uint8  
 15  관측소지점_152  51855 non-null  uint8  
 16  관측소지점_156  51855 non-null  uint8  
 17  관측소지점_159  51855 non-null  uint8  
 18  관측소지점_165  51855 non-null  uint8  
 19  지면타입_0     51855 non-null  uint8  
 20  지면타입_2

In [ ]:
X = df_f.drop(['자외선지수'], axis=1)
y = df_f['자외선지수']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 53)

In [ ]:
param_value = {'alpha' : [0.0001, 0.01, 1, 2, 3, 4]}
modelLasso = Lasso()
gridSearch = GridSearchCV(modelLasso, param_grid = param_value, cv = 10)
gridSearch.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.365e+03, tolerance: 1.660e+02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.370e+03, tolerance: 1.659e+02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisa

GridSearchCV(cv=10, estimator=Lasso(),
             param_grid={'alpha': [0.0001, 0.01, 1, 2, 3, 4]})

In [ ]:
gridSearch.best_params_

{'alpha': 0.0001}

In [ ]:
gridSearch.best_score_

0.8586740614367837

In [ ]:
gridSearch.cv_results_

{'mean_fit_time': array([24.8274704 ,  2.18180795,  0.20566647,  0.20360227,  0.22663343,
         0.22957563]),
 'mean_score_time': array([0.00911279, 0.00803547, 0.00767031, 0.00743709, 0.00867712,
        0.00792007]),
 'mean_test_score': array([0.85867406, 0.85188539, 0.64875627, 0.64647099, 0.64266098,
        0.63732625]),
 'param_alpha': masked_array(data=[0.0001, 0.01, 1, 2, 3, 4],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'alpha': 0.0001},
  {'alpha': 0.01},
  {'alpha': 1},
  {'alpha': 2},
  {'alpha': 3},
  {'alpha': 4}],
 'rank_test_score': array([1, 2, 3, 4, 5, 6], dtype=int32),
 'split0_test_score': array([0.85808485, 0.85130202, 0.64759645, 0.64523335, 0.64132818,
        0.63588092]),
 'split1_test_score': array([0.85925581, 0.8524225 , 0.64827716, 0.64606158, 0.64233577,
        0.63709974]),
 'split2_test_score': array([0.85940007, 0.85300707, 0.64789245, 0.64582635, 0.64222432,
      

In [ ]:
model = Lasso(alpha=0.0001)
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.041e+04, tolerance: 1.843e+02
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


Lasso(alpha=0.0001)

In [ ]:
y_pred = model.predict(X_test)
score1 = {'R2_score' : r2_score(y_test, y_pred)}
score2 = {'RMSE' : mean_squared_error(y_pred, y_test, squared = False)}

In [ ]:
score1

{'R2_score': 0.8585910086509569}

In [ ]:
score2

{'RMSE': 0.9699275486771961}

In [ ]:
test = df_d.drop(['자외선지수'], axis=1)

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51855 entries, 0 to 51854
Data columns (total 26 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   태양천정각      51855 non-null  float64
 1   Band1      51855 non-null  float64
 2   Band2      51855 non-null  float64
 3   관측소지점_13   51855 non-null  uint8  
 4   관측소지점_105  51855 non-null  uint8  
 5   관측소지점_108  51855 non-null  uint8  
 6   관측소지점_112  51855 non-null  uint8  
 7   관측소지점_115  51855 non-null  uint8  
 8   관측소지점_131  51855 non-null  uint8  
 9   관측소지점_132  51855 non-null  uint8  
 10  관측소지점_133  51855 non-null  uint8  
 11  관측소지점_138  51855 non-null  uint8  
 12  관측소지점_143  51855 non-null  uint8  
 13  관측소지점_146  51855 non-null  uint8  
 14  관측소지점_152  51855 non-null  uint8  
 15  관측소지점_156  51855 non-null  uint8  
 16  관측소지점_159  51855 non-null  uint8  
 17  관측소지점_165  51855 non-null  uint8  
 18  지면타입_0     51855 non-null  uint8  
 19  지면타입_2     51855 non-null  uint8  
 20  지면타입_3

In [ ]:
pred = model.predict(test)

In [ ]:
test['자외선지수'] = pred

In [ ]:
len(test)

51855

In [ ]:
test[(test['자외선지수'] < 0) & (test['time_3'] == 1)]

,태양천정각,Band1,Band2,관측소지점_13,관측소지점_105,관측소지점_108,관측소지점_112,관측소지점_115,관측소지점_131,관측소지점_132,관측소지점_133,관측소지점_138,관측소지점_143,관측소지점_146,관측소지점_152,관측소지점_156,관측소지점_159,관측소지점_165,지면타입_0,지면타입_2,지면타입_3,지면타입_4,time_1,time_2,time_3,Band345,자외선지수
0,123.72706,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,248.532806,-1.158482
1,119.74915,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,255.240625,-0.670674
2,119.65871,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,261.731743,-0.984628
3,119.71313,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,262.547848,-0.567339
4,120.21661,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,249.613523,-0.829469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51850,119.84431,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,271.610746,-0.731394
51851,120.42121,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,278.009191,-0.568305
51852,120.46896,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,274.867638,-0.534666
51853,120.85844,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,271.918007,-0.336475


,태양천정각,Band1,Band2,관측소지점_13,관측소지점_105,관측소지점_108,관측소지점_112,관측소지점_115,관측소지점_131,관측소지점_132,관측소지점_133,관측소지점_138,관측소지점_143,관측소지점_146,관측소지점_152,관측소지점_156,관측소지점_159,관측소지점_165,지면타입_0,지면타입_2,지면타입_3,지면타입_4,time_1,time_2,time_3,Band345,자외선지수
0,123.72706,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,248.532806,-1.158482
1,119.74915,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,255.240625,-0.670674
2,119.65871,0.0,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,261.731743,-0.984628
3,119.71313,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,262.547848,-0.567339
4,120.21661,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,249.613523,-0.829469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51850,119.84431,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,271.610746,-0.731394
51851,120.42121,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,278.009191,-0.568305
51852,120.46896,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,274.867638,-0.534666
51853,120.85844,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,271.918007,-0.336475


In [ ]:
mean_absolute_error(y_test, y_pred)

0.7359196654068525